# **1-Installation & Setup**

In [1]:
%%capture
!pip install kagglehub torchgeometry tqdm matplotlib
!pip install torch torchvision opencv-python
!pip install tensorboardX

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# **2-Kaggle Integration**

In [ ]:
# Upload kaggle.json
from google.colab import files
uploaded = files.upload()

In [4]:
# Set up Kaggle credentials
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
import kagglehub
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Download and unzip datasets
!kaggle datasets download -d marquis03/high-resolution-viton-zalando-dataset -p /content/data --unzip

# **3-Directory Preparation**

In [7]:
# Create test directories
os.makedirs("/content/data/test/image", exist_ok=True)
os.makedirs("/content/data/test/cloth", exist_ok=True)

# **4-Repository Cloning**

In [8]:
!rm -rf /content/HR-VITON

In [ ]:
!git clone https://github.com/sangyun884/HR-VITON.git

In [ ]:
# Verify test_generator.py exists
print("\nChecking for test_generator.py:")
!ls -lh /content/HR-VITON/test_generator.py

# **5-Dataset Handling**

In [11]:
# Create pairs file
image = ["00013_00.jpg", "00017_00.jpg"]
clothes = ["00006_00.jpg", "00008_00.jpg"]

pd.DataFrame({"image":image, "clothes":clothes}).to_csv("/content/data/pairs1.txt", index=False, header=False, sep=" ")

# **6-Model Weights**

In [ ]:
!kaggle datasets download -d marquis03/hr-viton -p /content/
!unzip /content/hr-viton.zip -d /content/HR-VITON/eval_models/weights/v0.1/

In [16]:
!mkdir -p /content/HR-VITON/eval_models/weights/v0.1/

In [ ]:
!cp /content/HR-VITON/eval_models/weights/v0.1/mtviton.pth /content/HR-VITON/eval_models/weights/v0.1/
!cp /content/HR-VITON/eval_models/weights/v0.1/gen.pth /content/HR-VITON/eval_models/weights/v0.1/

In [ ]:
!ls -lh /content/HR-VITON/eval_models/weights/v0.1/

# **7-Code Modifications**

In [22]:
!sed -i 's/\.astype(np\.float)/\.astype(float)/g' /content/HR-VITON/test_generator.py
!sed -i 's/\.astype(np\.float)/\.astype(float)/g' /content/HR-VITON/networks.py 2>/dev/null || echo "No networks.py found"

In [ ]:
!grep -n "astype(float)" /content/HR-VITON/test_generator.py

# **8-Execution**

In [ ]:
!cd /content/HR-VITON && python test_generator.py \
    --dataroot /content/data \
    --data_list /content/data/pairs1.txt \
    --output_dir /content/output \
    --tocg_checkpoint /content/HR-VITON/eval_models/weights/v0.1/mtviton.pth \
    --gen_checkpoint /content/HR-VITON/eval_models/weights/v0.1/gen.pth \
    --gpu_ids 0 \
    --occlusion

# **9-Visualization**

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

plt.figure(figsize=(12, 6*len(image)))
grid = plt.GridSpec(len(image), 3, wspace=0.1, hspace=0.3)

for idx in tqdm(range(len(image))):
    try:
        img_ori = plt.imread(f"/content/data/test/image/{image[idx]}")
        cloth = plt.imread(f"/content/data/test/cloth/{clothes[idx]}")

        base_name = f"{image[idx].split('.')[0]}_{clothes[idx].split('.')[0]}"
        output_files = [f for f in os.listdir("/content/output") if f.startswith(base_name)]

        if not output_files:
            raise FileNotFoundError(f"No output file found for {base_name}")

        output_path = f"/content/output/{output_files[0]}"
        try:
            img_new = plt.imread(output_path)
        except:
            img_new = np.array(Image.open(output_path))

        plt.subplot(grid[idx, 0])
        plt.imshow(img_ori)
        plt.axis("off")
        plt.title("Original Person", pad=10)

        plt.subplot(grid[idx, 1])
        plt.imshow(cloth)
        plt.axis("off")
        plt.title("Clothing Item", pad=10)

        plt.subplot(grid[idx, 2])
        plt.imshow(img_new)
        plt.axis("off")
        plt.title("Generated", pad=10)

    except Exception as e:
        print(f"\nError processing {image[idx]}: {str(e)}")
        continue

plt.tight_layout()
plt.show()